In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Constants
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN
rectangle_bgr = (255, 255, 255)
emotion_dict = {
    0: ("Angry", (0, 0, 255)),
    1: ("Disgust", (0, 0, 255)),
    2: ("Fear", (0, 0, 255)),
    3: ("Happy", (0, 0, 255)),
    4: ("Sad", (0, 0, 255)),
    5: ("Surprise", (0, 0, 255)),
    6: ("Neutral", (0, 255, 0))
}

# Load the face detection model
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the emotion recognition model
new_model = tf.keras.models.load_model('fer_model.h5')

# Open the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        face_roi = cv2.resize(roi_color, (224, 224))
        face_roi = np.expand_dims(face_roi, axis=0) / 255.0
        
        predictions = new_model.predict(face_roi)
        max_index = np.argmax(predictions)
        status, color = emotion_dict[max_index]

        x1, y1, w1, h1 = 0, 0, 175, 75
        cv2.rectangle(frame, (x1, y1), (x1 + w1, y1 + h1), (0, 0, 0), -1)
        cv2.putText(frame, status, (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, status, (x + 10, y - 10), font, font_scale, color, 2, cv2.LINE_AA)

    cv2.imshow("Facial Emotion Recognition", frame)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
